In [1]:
# Title:   Python Regression Examples
# Author:  Peter Scarbrough
# Date:    25 Jan 2020
# Purpose: Practice regression in Python, leading to pipeline creation

In [2]:
# 1. Load required packages
import math
import numpy as np
import pandas as pd
import sklearn.datasets as datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [3]:
# 2. Load, tidy data (boston housing)
bdata = datasets.load_boston()

# convert to pandas dataframe
b_df = pd.DataFrame(bdata.data, columns=bdata.feature_names)
b_df["MEDV"] = bdata.target

# print data frame to inspect
# note: skipping EDA for this exercise
#       but pandas df usually a good starting point for that
print(b_df)

        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0    0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1    0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2    0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3    0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4    0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...    ...   
501  0.06263   0.0  11.93   0.0  0.573  6.593  69.1  2.4786  1.0  273.0   
502  0.04527   0.0  11.93   0.0  0.573  6.120  76.7  2.2875  1.0  273.0   
503  0.06076   0.0  11.93   0.0  0.573  6.976  91.0  2.1675  1.0  273.0   
504  0.10959   0.0  11.93   0.0  0.573  6.794  89.3  2.3889  1.0  273.0   
505  0.04741   0.0  11.93   0.0  0.573  6.030  80.8  2.5050  1.0  273.0   

     PTRATIO       B  LSTAT  MEDV  
0       15.3  396.90   4.98  24.0  
1       17.8  396.90   9.14

In [4]:
# 3. linear regression modeling - include all predictors
#    note: this model is overfitted

# get features, response variable
X = b_df.iloc[:,:13]
y = b_df["MEDV"]

# use sklearn.linear_model.LinearRegression
lr    = LinearRegression()
model = lr.fit(X, y)

# get model R2
r2 = model.score(X, y)
print("R2 from overfit linear model: ", r2)

R2 from overfit linear model:  0.7406426641094095


### Comments:   
Lack of proper model selection strategies make this R2 an overestimate of the model's generalizability. Including all predictor terms in linear regression likely resulted in an overfit model. However, there is still value in taking a rough look at this statistic because 1) it demonstrates the process of performing a simple linear regresison in Python and 2) it gives us a rough idea of the amount of information content in the data that can be used to model the response.  

One way to avoid overfitting the data is to introduce regularization. Lasso is a method that will do this in linear regression while also potentially performing a kind of automated feature selection. This is illustrated below.

In [5]:
# 4. Linear Regression w/Regularization - Lasso Method

# scale X (mean = 0, std = 1)
scaler = StandardScaler()
stdX   = scaler.fit_transform(X)

# do lasso regression
lr_lasso    = Lasso(alpha=1)
model_lasso = lr_lasso.fit(stdX, y) 

# get model R2
r2_lasso = model_lasso.score(stdX, y)
print("R2 from naive lasso model: ", r2_lasso)

R2 from naive lasso model:  0.6628192101128285


### Comments :  
Although the score appears lower than the previous linear regression model, it is actually probably an improvement. However, to further improve the model's generalizability it is important to introduce proper model selection controls.

Splitting the data into test and training subsets, and performing model selection on the training subset and final model testing on the test subset should help reduce selection bias and improve the R2 estimate and better reflect the true generalizability of the model. Additionally, the lasso method has a hyperparameter (alpha) that controls the amount of shrinkage in the model. By using cross-validation one can select an optimal hyperparameter and further improve the generalizable performance of the resulting lasso model.

In [6]:
# 5. optimize lasso method (also a kind of feature selection)

# split into training and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

# create a pipeline for model selection 
# include feature selection (alpha) and standardization (required for lasso)
pipeline      = Pipeline([("standardize", StandardScaler()),
                          ("classifier", Lasso())])
alpha_space   = np.logspace(-3, 3, 30)
hyperp        = [{"classifier__alpha": alpha_space}]
lasso_modeler = GridSearchCV(pipeline, hyperp, cv=10, verbose=0, iid=False)
lasso_fit     = lasso_modeler.fit(X_train, y_train)

# get optimal alpha estimate from test data using lasso modeling object
optimal_alpha = lasso_fit.best_params_['classifier__alpha']
print("Optimal alpha for lasso model: ", optimal_alpha)

# then use the optimal alpha to build the final model
# use the test data to test that model and get an R2 estimate
lasso_modeler = Lasso(alpha=optimal_alpha)
lasso_fit     = lasso_modeler.fit(X_train, y_train)
lasso_fit_r2  = lasso_fit.score(X_test, y_test)
print("Optimal Lasso R2 estimate: ", lasso_fit_r2)

Optimal alpha for lasso model:  0.02807216203941177
Optimal Lasso R2 estimate:  0.6094753889003006


### Comments:  
Although this is the lowest reported R2 estimate yet, due to proper model selection controls being utilized, this is probably the most generalizable and believable estimate yet produced. There is probably still room for improvement. And we can do this, still working within a regression framework, by considering ridge regression as an alternative candidate model.

This will be explored, below, using a pipeline to control the standardization of the data, hyperparamter tuning, and model selection all in one system. The resulting model will then be evaluated with the test data to estimate R2. Note: GridSearchCV is used to handle the cross-validation system in Python for hyperparameter tuning. Once the hyperparameter is found the final model is refit using the regular method with the full training data.

In [7]:
# 6. Improve pipeline, add ridge regression as candidate model
#    test lasso and ridge while tuning hyperparameters with standardization
#    also include option to allow parallelization of calculation

# note: feature selection didn't work with lasso optimization
#       lack of zeroes suggests no automatic feature selection
print("lasso model coefficients: ", lasso_fit.coef_, "", sep="\n") 

# so it's worth trying ridge regression as well. let's incorporate into pipeline...

# split into training and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

# create a pipeline for model selection 
# include feature selection (alpha) and standardization (required for lasso)
pipeline      = Pipeline([("standardize", StandardScaler()),
                          ("classifier", Ridge())])
alpha_space    = np.logspace(-3, 3, 30)
hyperp         = [{"classifier": [Lasso()],
                   "classifier__alpha": alpha_space},
                  {"classifier": [Ridge()],
                   "classifier__alpha": alpha_space}]
linear_modeler = GridSearchCV(pipeline, hyperp, cv=10, verbose=0, iid=False, n_jobs=-1)
linear_fit     = linear_modeler.fit(X_train, y_train)

# get best model information
print("Best model information: ", linear_fit.best_params_['classifier'], "", sep="\n")

# get optimal alpha estimate for ridge model
optimal_alpha_ridge = linear_fit.best_params_['classifier__alpha']

# then use the optimal alpha to build the final model
# use the test data to test that model and get an R2 estimate
ridge_modeler = Ridge(alpha=optimal_alpha_ridge)
ridge_fit     = ridge_modeler.fit(X_train, y_train)
ridge_fit_r2  = ridge_fit.score(X_test, y_test)
print("Optimal Ridge R2 Estimate: ", ridge_fit_r2)

lasso model coefficients: 
[-1.02536033e-01  5.16042317e-02 -5.22340921e-03  2.46836096e+00
 -7.05449850e+00  4.58532400e+00 -1.41512085e-02 -1.18642856e+00
  2.34439987e-01 -1.19731189e-02 -8.86444294e-01  8.74788492e-03
 -4.59145631e-01]

Best model information: 
Ridge(alpha=13.738237958832638, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

Optimal Ridge R2 Estimate:  0.7265246392769191


# Comments

In this example, we find that ridge regression was better at preventing model overfit than the lasso method. The final R2 is an improvement upon earlier models while performing adequate model selection controls; thus it is a believable and good first step towards developing a good machine learning approach to this prediction problem.

In general for these two shrinkage methods (lasso and ridge), lasso is generally preferred for larger datasets and in cases where feature selection is desired, since it has the capacity to naturally eliminate candidate features. Lasso also tends to perform better in cases where there may be more outliers that are causes overfitting of the data. However, in simpler cases, it is not uncommon to see ridge regression outperform, as it tends to be generally better than lasso at reducing model overfit. Therefore, the result we found here wasn't too surprising. 